In [45]:
import pandas as pd
import numpy as np
import re
import nltk
import spacy
import string
import preprocessing_functions

#Text pre-processing

In [46]:
!cp gdrive/MyDrive/GitHub/NLP-project-/preprocessing_functions.py .

cp: cannot stat 'gdrive/MyDrive/GitHub/NLP-project-/preprocessing_functions.py': No such file or directory


In [47]:
!python preprocessing_functions.py 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [48]:
execfile('preprocessing_functions.py')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [49]:
#df = pd.read_csv('NLP_project_kickoff_data.csv', encoding='utf-8')
df = pd.read_csv('Instagram.csv', encoding='utf-8')
df_emo= pd.read_csv('emojis.csv')

In [50]:
percent_missing = df.isnull().sum() * 100 / len(df)
missing_value_df = pd.DataFrame({'column_name': df.columns,'percent_missing': percent_missing})
missing_value_df

,column_name,percent_missing
Unnamed: 0,Unnamed: 0,0.0
id,id,0.0
biography,biography,0.0
external_url,external_url,0.0
followers,followers,0.0
posts_count,posts_count,0.0
following,following,0.0
account,account,0.0
caption,caption,0.0
profile_name,profile_name,0.0


In [51]:
df.dtypes

Unnamed: 0              int64
id                    float64
biography              object
external_url           object
followers             float64
posts_count           float64
following             float64
account                object
caption                object
profile_name           object
profile_image_link     object
likes                 float64
datetime              float64
image_url              object
url                    object
comments              float64
thumbnails             object
input_account          object
dtype: object

In [52]:
string_columns=['biography', 'caption']

##Emoji replacement 

In [53]:
for i in string_columns:
  emojis_replacement(df[i])  

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


##Creation of Hashtags

In [54]:
df["hashtags"] = df.caption.apply(lambda x: [x for x in x.split(" ") if x.startswith("#")])

##Text Lower

In [55]:
df.apply(lambda x: x.astype(str).str.lower())
df

,Unnamed: 0,id,biography,external_url,followers,posts_count,following,account,caption,profile_name,profile_image_link,likes,datetime,image_url,url,comments,thumbnails,input_account,hashtags
0,29,2.770000e+18,Have your BBL with an authentic expertNflag4 \...,http://www.drjoffily.co.uk/,23300.0,276.0,7003.0,maurino_joffily,Quick recovery and result after 10 days only #...,Dr. Maurino Joffily,https://instagram.fwzy1-2.fna.fbcdn.net/v/t51....,173.0,1.640000e+12,https://instagram.fuvf1-1.fna.fbcdn.net/v/t51....,https://www.instagram.com/p/CZki1wfvXYq/,17.0,"[{""src"":""https://instagram.fuvf1-1.fna.fbcdn.n...",maurino_joffily,"[#maurinojoffily, #drmaurino, #jÃ³ffily, #drjo..."
1,30,2.760000e+18,Have your BBL with an authentic expertNflag4 \...,http://www.drjoffily.co.uk/,23300.0,276.0,7003.0,maurino_joffily,This patient is very happy with this change on...,Dr. Maurino Joffily,https://instagram.fwzy1-2.fna.fbcdn.net/v/t51....,388.0,1.640000e+12,https://instagram.fuvf1-1.fna.fbcdn.net/v/t51....,https://www.instagram.com/p/CZUrYFrOO5v/,24.0,"[{""src"":""https://instagram.fuvf1-1.fna.fbcdn.n...",maurino_joffily,"[#maurinojoffily, #drmaurino, #drjoffily, #drj..."
2,31,2.760000e+18,Have your BBL with an authentic expertNflag4 \...,http://www.drjoffily.co.uk/,23300.0,276.0,7003.0,maurino_joffily,360 Vaser lipo and fat transfer to correct a h...,Dr. Maurino Joffily,https://instagram.fwzy1-2.fna.fbcdn.net/v/t51....,160.0,1.640000e+12,https://instagram.fuvf1-1.fna.fbcdn.net/v/t51....,https://www.instagram.com/p/CZAgVoyPi0i/,8.0,"[{""src"":""https://instagram.fuvf1-1.fna.fbcdn.n...",maurino_joffily,"[#maurino, #joffily, #joffilyvaser, #drmaurino..."
3,32,2.760000e+18,Have your BBL with an authentic expertNflag4 \...,http://www.drjoffily.co.uk/,23300.0,276.0,7003.0,maurino_joffily,Breast uplift without implants is a good combi...,Dr. Maurino Joffily,https://instagram.fwzy1-2.fna.fbcdn.net/v/t51....,211.0,1.640000e+12,https://instagram.fuvf1-1.fna.fbcdn.net/v/t51....,https://www.instagram.com/p/CY9pMGVP_BY/,24.0,"[{""src"":""https://instagram.fuvf1-1.fna.fbcdn.n...",maurino_joffily,"[#drjoffily, #maurino, #maurinojoffily, #vaser..."
4,33,2.750000e+18,Have your BBL with an authentic expertNflag4 \...,http://www.drjoffily.co.uk/,23300.0,276.0,7003.0,maurino_joffily,Great transformation after these combined proc...,Dr. Maurino Joffily,https://instagram.fwzy1-2.fna.fbcdn.net/v/t51....,210.0,1.640000e+12,https://instagram.fuvf1-1.fna.fbcdn.net/v/t51....,https://www.instagram.com/p/CYrbwXQrY3v/,15.0,"[{""src"":""https://instagram.fuvf1-1.fna.fbcdn.n...",maurino_joffily,"[#drmaurino, #maurino, #drjoffily, #vaserlipo,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58924,62916,1.630000e+18,"Consultant Plastic, Aesthetic & Reconstructive...",https://linktr.ee/drallenrezai,155118.0,776.0,376.0,drallenrezai,See the good in people and situations and your...,Allen Rezai MD,https://scontent-iev1-1.cdninstagram.com/v/t51...,102.0,1.510000e+12,https://instagram.ftas2-1.fna.fbcdn.net/v/t51....,https://www.instagram.com/p/BawCgfjDtqX/,1.0,"[{""src"":""https://instagram.ftas2-1.fna.fbcdn.n...",drallenrezai,[]
58925,62917,1.630000e+18,"Consultant Plastic, Aesthetic & Reconstructive...",https://linktr.ee/drallenrezai,155118.0,776.0,376.0,drallenrezai,ExpatWoman's Nese had the opportunity to try o...,Allen Rezai MD,https://scontent-iev1-1.cdninstagram.com/v/t51...,131.0,1.510000e+12,https://instagram.ftas1-2.fna.fbcdn.net/v/t51....,https://www.instagram.com/p/BaqhugBDQvU/,6.0,"[{""src"":""https://instagram.ftas1-2.fna.fbcdn.n...",drallenrezai,[]
58926,62918,1.630000e+18,"Consultant Plastic, Aesthetic & Reconstructive...",https://linktr.ee/drallenrezai,155118.0,776.0,376.0,drallenrezai,The decision to have Breast Implant Surgery is...,Allen Rezai MD,https://scontent-iev1-1.cdninstagram.com/v/t51...,115.0,1.510000e+12,https://instagram.ftas1-2.fna.fbcdn.net/v/t51....,https://www.instagram.com/p/BalNRWQjn7g/,1.0,"[{""src"":""https://

##Remove punctuation

In [56]:
df["biography_wo_punc"] = df["biography"].str.replace('[{}]'.format(string.punctuation), '')
df["caption_wo_punc"] = df["caption"].str.replace('[{}]'.format(string.punctuation), '')
df["profile_wo_punc"] = df["profile_name"].str.replace('[{}]'.format(string.punctuation), '')
# df["tokens_wo_punc"] = df["tokens"].str.replace('[{}]'.format(string.punctuation), '')

df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


,Unnamed: 0,id,biography,external_url,followers,posts_count,following,account,caption,profile_name,profile_image_link,likes,datetime,image_url,url,comments,thumbnails,input_account,hashtags,biography_wo_punc,caption_wo_punc,profile_wo_punc
0,29,2.770000e+18,Have your BBL with an authentic expertNflag4 \...,http://www.drjoffily.co.uk/,23300.0,276.0,7003.0,maurino_joffily,Quick recovery and result after 10 days only #...,Dr. Maurino Joffily,https://instagram.fwzy1-2.fna.fbcdn.net/v/t51....,173.0,1.640000e+12,https://instagram.fuvf1-1.fna.fbcdn.net/v/t51....,https://www.instagram.com/p/CZki1wfvXYq/,17.0,"[{""src"":""https://instagram.fuvf1-1.fna.fbcdn.n...",maurino_joffily,"[#maurinojoffily, #drmaurino, #jÃ³ffily, #drjo...",Have your BBL with an authentic expertNflag4 \...,Quick recovery and result after 10 days only m...,Dr Maurino Joffily
1,30,2.760000e+18,Have your BBL with an authentic expertNflag4 \...,http://www.drjoffily.co.uk/,23300.0,276.0,7003.0,maurino_joffily,This patient is very happy with this change on...,Dr. Maurino Joffily,https://instagram.fwzy1-2.fna.fbcdn.net/v/t51....,388.0,1.640000e+12,https://instagram.fuvf1-1.fna.fbcdn.net/v/t51....,https://www.instagram.com/p/CZUrYFrOO5v/,24.0,"[{""src"":""https://instagram.fuvf1-1.fna.fbcdn.n...",maurino_joffily,"[#maurinojoffily, #drmaurino, #drjoffily, #drj...",Have your BBL with an authentic expertNflag4 \...,This patient is very happy with this change on...,Dr Maurino Joffily
2,31,2.760000e+18,Have your BBL with an authentic expertNflag4 \...,http://www.drjoffily.co.uk/,23300.0,276.0,7003.0,maurino_joffily,360 Vaser lipo and fat transfer to correct a h...,Dr. Maurino Joffily,https://instagram.fwzy1-2.fna.fbcdn.net/v/t51....,160.0,1.640000e+12,https://instagram.fuvf1-1.fna.fbcdn.net/v/t51....,https://www.instagram.com/p/CZAgVoyPi0i/,8.0,"[{""src"":""https://instagram.fuvf1-1.fna.fbcdn.n...",maurino_joffily,"[#maurino, #joffily, #joffilyvaser, #drmaurino...",Have your BBL with an authentic expertNflag4 \...,360 Vaser lipo and fat transfer to correct a h...,Dr Maurino Joffily
3,32,2.760000e+18,Have your BBL with an authentic expertNflag4 \...,http://www.drjoffily.co.uk/,23300.0,276.0,7003.0,maurino_joffily,Breast uplift without implants is a good combi...,Dr. Maurino Joffily,https://instagram.fwzy1-2.fna.fbcdn.net/v/t51....,211.0,1.640000e+12,https://instagram.fuvf1-1.fna.fbcdn.net/v/t51....,https://www.instagram.com/p/CY9pMGVP_BY/,24.0,"[{""src"":""https://instagram.fuvf1-1.fna.fbcdn.n...",maurino_joffily,"[#drjoffily, #maurino, #maurinojoffily, #vaser...",Have your BBL with an authentic expertNflag4 \...,Breast uplift without implants is a good combi...,Dr Maurino Joffily
4,33,2.750000e+18,Have your BBL with an authentic expertNflag4 \...,http://www.drjoffily.co.uk/,23300.0,276.0,7003.0,maurino_joffily,Great transformation after these combined proc...,Dr. Maurino Joffily,https://instagram.fwzy1-2.fna.fbcdn.net/v/t51....,210.0,1.640000e+12,https://instagram.fuvf1-1.fna.fbcdn.net/v/t51....,https://www.instagram.com/p/CYrbwXQrY3v/,15.0,"[{""src"":""https://instagram.fuvf1-1.fna.fbcdn.n...",maurino_joffily,"[#drmaurino, #maurino, #drjoffily, #vaserlipo,...",Have your BBL with an authentic expertNflag4 \...,Great transformation after these combined proc...,Dr Maurino Joffily
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58924,62916,1.630000e+18,"Consultant Plastic, Aesthetic & Reconstructive...",https://linktr.ee/drallenrezai,155118.0,776.0,376.0,drallenrezai,See the good in people and situations and your...,Allen Rezai MD,https://scontent-iev1-1.cdninstagram.com/v/t51...,102.0,1.510000e+12,https://instagram.ftas2-1.fna.fbcdn.net/v/t51....,https://www.instagram.com/p/BawCgfjDtqX/,1.0,"[{""src"":""https://instagram.ftas2-1.fna.fbcdn.n...",drallenrezai,[],Consultant Plastic Aesthetic Reconstructive S...,See the good in people and situations and your...,Allen Rezai MD
58925,62917,1.630000e+18,"Consultant Plastic, Aesthetic & Recon

In [57]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [58]:
df["biography_wo_stop"] = df["biography_wo_punc"].apply(lambda text: remove_stopwords(text))
df["caption_wo_stop"] = df["caption_wo_punc"].apply(lambda text: remove_stopwords(text))
df["profile_wo_stop"] = df["profile_wo_punc"].apply(lambda text: remove_stopwords(text))
# df["tokens_wo_stop"] = df["tokens_wo_punc"].apply(lambda text: remove_stopwords(text))


##Removal of frequent words

In [59]:
from collections import Counter
cnt = Counter()
for text in df["biography_wo_stop"].values:
    for word in text.split():
        cnt[word] += 1
        
cnt.most_common(10)

[('Plastic', 11885),
 ('Surgeon', 11132),
 ('Nsign16', 9836),
 ('London', 9159),
 ('Cosmetic', 8218),
 ('Surgery', 7308),
 ('Lneedle', 5593),
 ('Lphone1', 4729),
 ('Aesthetic', 4728),
 ('UK', 4290)]

In [60]:
FREQWORDS = set([w for (w, wc) in cnt.most_common(10)])
# def remove_freqwords(text):
#     """custom function to remove the frequent words"""
#     return " ".join([word for word in str(text).split() if word not in FREQWORDS])

df["biography_wo_stopfreq"] = df["biography_wo_stop"].apply(lambda text: remove_freqwords(text))
df["caption_wo_stopfreq"] = df["caption_wo_stop"].apply(lambda text: remove_freqwords(text))
df["profile_wo_stopfreq"] = df["profile_wo_stop"].apply(lambda text: remove_freqwords(text))
# df["tokens_wo_stopfreq"] = df["tokens_wo_stop"].apply(lambda text: remove_freqwords(text))

df.head()

,Unnamed: 0,id,biography,external_url,followers,posts_count,following,account,caption,profile_name,profile_image_link,likes,datetime,image_url,url,comments,thumbnails,input_account,hashtags,biography_wo_punc,caption_wo_punc,profile_wo_punc,biography_wo_stop,caption_wo_stop,profile_wo_stop,biography_wo_stopfreq,caption_wo_stopfreq,profile_wo_stopfreq
0,29,2.770000e+18,Have your BBL with an authentic expertNflag4 \...,http://www.drjoffily.co.uk/,23300.0,276.0,7003.0,maurino_joffily,Quick recovery and result after 10 days only #...,Dr. Maurino Joffily,https://instagram.fwzy1-2.fna.fbcdn.net/v/t51....,173.0,1.640000e+12,https://instagram.fuvf1-1.fna.fbcdn.net/v/t51....,https://www.instagram.com/p/CZki1wfvXYq/,17.0,"[{""src"":""https://instagram.fuvf1-1.fna.fbcdn.n...",maurino_joffily,"[#maurinojoffily, #drmaurino, #jÃ³ffily, #drjo...",Have your BBL with an authentic expertNflag4 \...,Quick recovery and result after 10 days only m...,Dr Maurino Joffily,Have BBL authentic expertNflag4 Specialised Va...,Quick recovery result 10 days maurinojoffily d...,Dr Maurino Joffily,Have BBL authentic expertNflag4 Specialised Va...,Quick recovery result 10 days maurinojoffily d...,Dr Maurino Joffily
1,30,2.760000e+18,Have your BBL with an authentic expertNflag4 \...,http://www.drjoffily.co.uk/,23300.0,276.0,7003.0,maurino_joffily,This patient is very happy with this change on...,Dr. Maurino Joffily,https://instagram.fwzy1-2.fna.fbcdn.net/v/t51....,388.0,1.640000e+12,https://instagram.fuvf1-1.fna.fbcdn.net/v/t51....,https://www.instagram.com/p/CZUrYFrOO5v/,24.0,"[{""src"":""https://instagram.fuvf1-1.fna.fbcdn.n...",maurino_joffily,"[#maurinojoffily, #drmaurino, #drjoffily, #drj...",Have your BBL with an authentic expertNflag4 \...,This patient is very happy with this change on...,Dr Maurino Joffily,Have BBL authentic expertNflag4 Specialised Va...,This patient happy change body totally new fig...,Dr Maurino Joffily,Have BBL authentic expertNflag4 Specialised Va...,This patient happy change body totally new fig...,Dr Maurino Joffily
2,31,2.760000e+18,Have your BBL with an authentic expertNflag4 \...,http://www.drjoffily.co.uk/,23300.0,276.0,7003.0,maurino_joffily,360 Vaser lipo and fat transfer to correct a h...,Dr. Maurino Joffily,https://instagram.fwzy1-2.fna.fbcdn.net/v/t51....,160.0,1.640000e+12,https://instagram.fuvf1-1.fna.fbcdn.net/v/t51....,https://www.instagram.com/p/CZAgVoyPi0i/,8.0,"[{""src"":""https://instagram.fuvf1-1.fna.fbcdn.n...",maurino_joffily,"[#maurino, #joffily, #joffilyvaser, #drmaurino...",Have your BBL with an authentic expertNflag4 \...,360 Vaser lipo and fat transfer to correct a h...,Dr Maurino Joffily,Have BBL authentic expertNflag4 Specialised Va...,360 Vaser lipo fat transfer correct hip dip ma...,Dr Maurino Joffily,Have BBL authentic expertNflag4 Specialised Va...,360 Vaser lipo fat transfer correct hip dip ma...,Dr Maurino Joffily
3,32,2.760000e+18,Have your BBL with an authentic expertNflag4 \...,http://www.drjoffily.co.uk/,23300.0,276.0,7003.0,maurino_joffily,Breast uplift without implants is a good combi...,Dr. Maurino Joffily,https://instagram.fwzy1-2.fna.fbcdn.net/v/t51....,211.0,1.640000e+12,https://instagram.fuvf1-1.fna.fbcdn.net/v/t51....,https://www.instagram.com/p/CY9pMGVP_BY/,24.0,"[{""src"":""https://instagram.fuvf1-1.fna.fbcdn.n...",maurino_joffily,"[#drjoffily, #maurino, #maurinojoffily, #vaser...",Have your BBL with an authentic expertNflag4 \...,Breast uplift without implants is a good combi...,Dr Maurino Joffily,Have BBL authentic expertNflag4 Specialised Va...,Breast uplift without implants good combinatio...,Dr Maurino Joffily,Have BBL authentic expertNflag4 Specialised Va...,Breast uplift without implants good combinatio...,Dr Maurino Joffily
4,33,2.750000e+18,Have your BBL with an authentic expertNflag4 \...,http://www.drjoffily.co.uk/,23300.0,276.0,7003.0,maurino_joffily,Great transformation after these combined proc...,Dr. Maurino Joffily,https://instagram.fwzy1-2.fna.fbcdn.net/v/t51....,210.

##removal of rare words

In [61]:

n_rare_words = 10
RAREWORDS = set([w for (w, wc) in cnt.most_common()[:-n_rare_words-1:-1]])



In [62]:
df["biography_wo_stopfreqrare"] = df["biography_wo_stopfreq"].apply(lambda text: remove_rarewords(text))
df["caption_wo_stopfreqrare"] = df["caption_wo_stopfreq"].apply(lambda text: remove_rarewords(text))
df["profile_wo_stopfreqrare"] = df["profile_wo_stopfreq"].apply(lambda text: remove_rarewords(text))
# df["tokens_wo_stopfreqrare"] = df["tokens_wo_stopfreq"].apply(lambda text: remove_rarewords(text))

df.head()

,Unnamed: 0,id,biography,external_url,followers,posts_count,following,account,caption,profile_name,profile_image_link,likes,datetime,image_url,url,comments,thumbnails,input_account,hashtags,biography_wo_punc,caption_wo_punc,profile_wo_punc,biography_wo_stop,caption_wo_stop,profile_wo_stop,biography_wo_stopfreq,caption_wo_stopfreq,profile_wo_stopfreq,biography_wo_stopfreqrare,caption_wo_stopfreqrare,profile_wo_stopfreqrare
0,29,2.770000e+18,Have your BBL with an authentic expertNflag4 \...,http://www.drjoffily.co.uk/,23300.0,276.0,7003.0,maurino_joffily,Quick recovery and result after 10 days only #...,Dr. Maurino Joffily,https://instagram.fwzy1-2.fna.fbcdn.net/v/t51....,173.0,1.640000e+12,https://instagram.fuvf1-1.fna.fbcdn.net/v/t51....,https://www.instagram.com/p/CZki1wfvXYq/,17.0,"[{""src"":""https://instagram.fuvf1-1.fna.fbcdn.n...",maurino_joffily,"[#maurinojoffily, #drmaurino, #jÃ³ffily, #drjo...",Have your BBL with an authentic expertNflag4 \...,Quick recovery and result after 10 days only m...,Dr Maurino Joffily,Have BBL authentic expertNflag4 Specialised Va...,Quick recovery result 10 days maurinojoffily d...,Dr Maurino Joffily,Have BBL authentic expertNflag4 Specialised Va...,Quick recovery result 10 days maurinojoffily d...,Dr Maurino Joffily,Have BBL authentic expertNflag4 Specialised Va...,Quick recovery result 10 days maurinojoffily d...,Dr Maurino Joffily
1,30,2.760000e+18,Have your BBL with an authentic expertNflag4 \...,http://www.drjoffily.co.uk/,23300.0,276.0,7003.0,maurino_joffily,This patient is very happy with this change on...,Dr. Maurino Joffily,https://instagram.fwzy1-2.fna.fbcdn.net/v/t51....,388.0,1.640000e+12,https://instagram.fuvf1-1.fna.fbcdn.net/v/t51....,https://www.instagram.com/p/CZUrYFrOO5v/,24.0,"[{""src"":""https://instagram.fuvf1-1.fna.fbcdn.n...",maurino_joffily,"[#maurinojoffily, #drmaurino, #drjoffily, #drj...",Have your BBL with an authentic expertNflag4 \...,This patient is very happy with this change on...,Dr Maurino Joffily,Have BBL authentic expertNflag4 Specialised Va...,This patient happy change body totally new fig...,Dr Maurino Joffily,Have BBL authentic expertNflag4 Specialised Va...,This patient happy change body totally new fig...,Dr Maurino Joffily,Have BBL authentic expertNflag4 Specialised Va...,This patient happy change body totally new fig...,Dr Maurino Joffily
2,31,2.760000e+18,Have your BBL with an authentic expertNflag4 \...,http://www.drjoffily.co.uk/,23300.0,276.0,7003.0,maurino_joffily,360 Vaser lipo and fat transfer to correct a h...,Dr. Maurino Joffily,https://instagram.fwzy1-2.fna.fbcdn.net/v/t51....,160.0,1.640000e+12,https://instagram.fuvf1-1.fna.fbcdn.net/v/t51....,https://www.instagram.com/p/CZAgVoyPi0i/,8.0,"[{""src"":""https://instagram.fuvf1-1.fna.fbcdn.n...",maurino_joffily,"[#maurino, #joffily, #joffilyvaser, #drmaurino...",Have your BBL with an authentic expertNflag4 \...,360 Vaser lipo and fat transfer to correct a h...,Dr Maurino Joffily,Have BBL authentic expertNflag4 Specialised Va...,360 Vaser lipo fat transfer correct hip dip ma...,Dr Maurino Joffily,Have BBL authentic expertNflag4 Specialised Va...,360 Vaser lipo fat transfer correct hip dip ma...,Dr Maurino Joffily,Have BBL authentic expertNflag4 Specialised Va...,360 Vaser lipo fat transfer correct hip dip ma...,Dr Maurino Joffily
3,32,2.760000e+18,Have your BBL with an authentic expertNflag4 \...,http://www.drjoffily.co.uk/,23300.0,276.0,7003.0,maurino_joffily,Breast uplift without implants is a good combi...,Dr. Maurino Joffily,https://instagram.fwzy1-2.fna.fbcdn.net/v/t51....,211.0,1.640000e+12,https://instagram.fuvf1-1.fna.fbcdn.net/v/t51....,https://www.instagram.com/p/CY9pMGVP_BY/,24.0,"[{""src"":""https://instagram.fuvf1-1.fna.fbcdn.n...",maurino_joffily,"[#drjoffily, #maurino, #maurinojoffily, #vaser...",Have your BBL with an authentic expertNflag4 \...,Breast uplift without implants is a good combi...,Dr Maurino Joffily,Have BBL authentic expertNflag4 Specialised Va...,Breast uplift 

##Stemming

In [63]:
# from nltk.stem.porter import PorterStemmer

# stemmer = PorterStemmer()
# def stem_words(text):
#     return " ".join([stemmer.stem(word) for word in text.split()])

In [64]:
df["biography_stemmed"] = df["biography_wo_stopfreqrare"].apply(lambda text: stem_words(text))
df["caption_stemmed"] = df["caption_wo_stopfreqrare"].apply(lambda text: stem_words(text))
# df["profile_stemmed"] = df["profile_wo_punc"].str.apply(lambda text: stem_words(text))
# df["tokens_stemmed"] = df["tokens_wo_punc"].apply(lambda text: stem_words(text))

##Lemmatization

In [65]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [66]:
from nltk.stem import WordNetLemmatizer

# lemmatizer = WordNetLemmatizer()
# def lemmatize_words(text):
#     return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

df["biography_final"] = df["biography_stemmed"].apply(lambda text: lemmatize_words(text))
df["caption_final"] = df["caption_stemmed"].apply(lambda text: lemmatize_words(text))
df["profile_final"] = df["profile_wo_stopfreqrare"].apply(lambda text: lemmatize_words(text))
# df["tokens_final"] = df["tokens_stemmed"].apply(lambda text: lemmatize_words(text))

df.head()

,Unnamed: 0,id,biography,external_url,followers,posts_count,following,account,caption,profile_name,profile_image_link,likes,datetime,image_url,url,comments,thumbnails,input_account,hashtags,biography_wo_punc,caption_wo_punc,profile_wo_punc,biography_wo_stop,caption_wo_stop,profile_wo_stop,biography_wo_stopfreq,caption_wo_stopfreq,profile_wo_stopfreq,biography_wo_stopfreqrare,caption_wo_stopfreqrare,profile_wo_stopfreqrare,biography_stemmed,caption_stemmed,biography_final,caption_final,profile_final
0,29,2.770000e+18,Have your BBL with an authentic expertNflag4 \...,http://www.drjoffily.co.uk/,23300.0,276.0,7003.0,maurino_joffily,Quick recovery and result after 10 days only #...,Dr. Maurino Joffily,https://instagram.fwzy1-2.fna.fbcdn.net/v/t51....,173.0,1.640000e+12,https://instagram.fuvf1-1.fna.fbcdn.net/v/t51....,https://www.instagram.com/p/CZki1wfvXYq/,17.0,"[{""src"":""https://instagram.fuvf1-1.fna.fbcdn.n...",maurino_joffily,"[#maurinojoffily, #drmaurino, #jÃ³ffily, #drjo...",Have your BBL with an authentic expertNflag4 \...,Quick recovery and result after 10 days only m...,Dr Maurino Joffily,Have BBL authentic expertNflag4 Specialised Va...,Quick recovery result 10 days maurinojoffily d...,Dr Maurino Joffily,Have BBL authentic expertNflag4 Specialised Va...,Quick recovery result 10 days maurinojoffily d...,Dr Maurino Joffily,Have BBL authentic expertNflag4 Specialised Va...,Quick recovery result 10 days maurinojoffily d...,Dr Maurino Joffily,have bbl authent expertnflag4 specialis vaserl...,quick recoveri result 10 day maurinojoffili dr...,have bbl authent expertnflag4 specialis vaserl...,quick recoveri result 10 day maurinojoffili dr...,Dr Maurino Joffily
1,30,2.760000e+18,Have your BBL with an authentic expertNflag4 \...,http://www.drjoffily.co.uk/,23300.0,276.0,7003.0,maurino_joffily,This patient is very happy with this change on...,Dr. Maurino Joffily,https://instagram.fwzy1-2.fna.fbcdn.net/v/t51....,388.0,1.640000e+12,https://instagram.fuvf1-1.fna.fbcdn.net/v/t51....,https://www.instagram.com/p/CZUrYFrOO5v/,24.0,"[{""src"":""https://instagram.fuvf1-1.fna.fbcdn.n...",maurino_joffily,"[#maurinojoffily, #drmaurino, #drjoffily, #drj...",Have your BBL with an authentic expertNflag4 \...,This patient is very happy with this change on...,Dr Maurino Joffily,Have BBL authentic expertNflag4 Specialised Va...,This patient happy change body totally new fig...,Dr Maurino Joffily,Have BBL authentic expertNflag4 Specialised Va...,This patient happy change body totally new fig...,Dr Maurino Joffily,Have BBL authentic expertNflag4 Specialised Va...,This patient happy change body totally new fig...,Dr Maurino Joffily,have bbl authent expertnflag4 specialis vaserl...,thi patient happi chang bodi total new figur a...,have bbl authent expertnflag4 specialis vaserl...,thi patient happi chang bodi total new figur a...,Dr Maurino Joffily
2,31,2.760000e+18,Have your BBL with an authentic expertNflag4 \...,http://www.drjoffily.co.uk/,23300.0,276.0,7003.0,maurino_joffily,360 Vaser lipo and fat transfer to correct a h...,Dr. Maurino Joffily,https://instagram.fwzy1-2.fna.fbcdn.net/v/t51....,160.0,1.640000e+12,https://instagram.fuvf1-1.fna.fbcdn.net/v/t51....,https://www.instagram.com/p/CZAgVoyPi0i/,8.0,"[{""src"":""https://instagram.fuvf1-1.fna.fbcdn.n...",maurino_joffily,"[#maurino, #joffily, #joffilyvaser, #drmaurino...",Have your BBL with an authentic expertNflag4 \...,360 Vaser lipo and fat transfer to correct a h...,Dr Maurino Joffily,Have BBL authentic expertNflag4 Specialised Va...,360 Vaser lipo fat transfer correct hip dip ma...,Dr Maurino Joffily,Have BBL authentic expertNflag4 Specialised Va...,360 Vaser lipo fat transfer correct hip dip ma...,Dr Maurino Joffily,Have BBL authentic expertNflag4 Specialised Va...,360 Vaser lipo fat transfer correct hip dip ma...,Dr Maurino Joffily,have bbl authent expertnflag4 specialis vaserl...,360 vaser lipo fat transfer correct hip dip ma...,have bbl authent expertnflag4 specialis vaserl...,360 vase

##Removal of URLs

In [67]:
df['biography_final'] = df['biography_final'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])
df['caption_final'] = df['caption_final'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])
df

,Unnamed: 0,id,biography,external_url,followers,posts_count,following,account,caption,profile_name,profile_image_link,likes,datetime,image_url,url,comments,thumbnails,input_account,hashtags,biography_wo_punc,caption_wo_punc,profile_wo_punc,biography_wo_stop,caption_wo_stop,profile_wo_stop,biography_wo_stopfreq,caption_wo_stopfreq,profile_wo_stopfreq,biography_wo_stopfreqrare,caption_wo_stopfreqrare,profile_wo_stopfreqrare,biography_stemmed,caption_stemmed,biography_final,caption_final,profile_final
0,29,2.770000e+18,Have your BBL with an authentic expertNflag4 \...,http://www.drjoffily.co.uk/,23300.0,276.0,7003.0,maurino_joffily,Quick recovery and result after 10 days only #...,Dr. Maurino Joffily,https://instagram.fwzy1-2.fna.fbcdn.net/v/t51....,173.0,1.640000e+12,https://instagram.fuvf1-1.fna.fbcdn.net/v/t51....,https://www.instagram.com/p/CZki1wfvXYq/,17.0,"[{""src"":""https://instagram.fuvf1-1.fna.fbcdn.n...",maurino_joffily,"[#maurinojoffily, #drmaurino, #jÃ³ffily, #drjo...",Have your BBL with an authentic expertNflag4 \...,Quick recovery and result after 10 days only m...,Dr Maurino Joffily,Have BBL authentic expertNflag4 Specialised Va...,Quick recovery result 10 days maurinojoffily d...,Dr Maurino Joffily,Have BBL authentic expertNflag4 Specialised Va...,Quick recovery result 10 days maurinojoffily d...,Dr Maurino Joffily,Have BBL authentic expertNflag4 Specialised Va...,Quick recovery result 10 days maurinojoffily d...,Dr Maurino Joffily,have bbl authent expertnflag4 specialis vaserl...,quick recoveri result 10 day maurinojoffili dr...,have bbl authent expertnflag4 specialis vaserl...,quick recoveri result 10 day maurinojoffili dr...,Dr Maurino Joffily
1,30,2.760000e+18,Have your BBL with an authentic expertNflag4 \...,http://www.drjoffily.co.uk/,23300.0,276.0,7003.0,maurino_joffily,This patient is very happy with this change on...,Dr. Maurino Joffily,https://instagram.fwzy1-2.fna.fbcdn.net/v/t51....,388.0,1.640000e+12,https://instagram.fuvf1-1.fna.fbcdn.net/v/t51....,https://www.instagram.com/p/CZUrYFrOO5v/,24.0,"[{""src"":""https://instagram.fuvf1-1.fna.fbcdn.n...",maurino_joffily,"[#maurinojoffily, #drmaurino, #drjoffily, #drj...",Have your BBL with an authentic expertNflag4 \...,This patient is very happy with this change on...,Dr Maurino Joffily,Have BBL authentic expertNflag4 Specialised Va...,This patient happy change body totally new fig...,Dr Maurino Joffily,Have BBL authentic expertNflag4 Specialised Va...,This patient happy change body totally new fig...,Dr Maurino Joffily,Have BBL authentic expertNflag4 Specialised Va...,This patient happy change body totally new fig...,Dr Maurino Joffily,have bbl authent expertnflag4 specialis vaserl...,thi patient happi chang bodi total new figur a...,have bbl authent expertnflag4 specialis vaserl...,thi patient happi chang bodi total new figur a...,Dr Maurino Joffily
2,31,2.760000e+18,Have your BBL with an authentic expertNflag4 \...,http://www.drjoffily.co.uk/,23300.0,276.0,7003.0,maurino_joffily,360 Vaser lipo and fat transfer to correct a h...,Dr. Maurino Joffily,https://instagram.fwzy1-2.fna.fbcdn.net/v/t51....,160.0,1.640000e+12,https://instagram.fuvf1-1.fna.fbcdn.net/v/t51....,https://www.instagram.com/p/CZAgVoyPi0i/,8.0,"[{""src"":""https://instagram.fuvf1-1.fna.fbcdn.n...",maurino_joffily,"[#maurino, #joffily, #joffilyvaser, #drmaurino...",Have your BBL with an authentic expertNflag4 \...,360 Vaser lipo and fat transfer to correct a h...,Dr Maurino Joffily,Have BBL authentic expertNflag4 Specialised Va...,360 Vaser lipo fat transfer correct hip dip ma...,Dr Maurino Joffily,Have BBL authentic expertNflag4 Specialised Va...,360 Vaser lipo fat transfer correct hip dip ma...,Dr Maurino Joffily,Have BBL authentic expertNflag4 Specialised Va...,360 Vaser lipo fat transfer correct hip dip ma...,Dr Maurino Joffily,have bbl authent expertnflag4 specialis vaserl...,360 vaser lipo fat transfer correct hip dip ma...,have bbl authent expertnflag4 specialis vaserl...,360 vase

#Final dataframe

In [68]:
df_final=df.drop(['biography','caption', 'profile_name', 'biography_wo_punc',	'caption_wo_punc'	,'biography_wo_stop',	'caption_wo_stop',	'profile_wo_punc'	,'profile_wo_stop'	,'biography_wo_stopfreq',	'caption_wo_stopfreq'	,'profile_wo_stopfreq','biography_wo_stopfreqrare'	,'caption_wo_stopfreqrare',	'profile_wo_stopfreqrare'	,'biography_stemmed'	,'caption_stemmed'], axis=1)
df_final

,Unnamed: 0,id,external_url,followers,posts_count,following,account,profile_image_link,likes,datetime,image_url,url,comments,thumbnails,input_account,hashtags,biography_final,caption_final,profile_final
0,29,2.770000e+18,http://www.drjoffily.co.uk/,23300.0,276.0,7003.0,maurino_joffily,https://instagram.fwzy1-2.fna.fbcdn.net/v/t51....,173.0,1.640000e+12,https://instagram.fuvf1-1.fna.fbcdn.net/v/t51....,https://www.instagram.com/p/CZki1wfvXYq/,17.0,"[{""src"":""https://instagram.fuvf1-1.fna.fbcdn.n...",maurino_joffily,"[#maurinojoffily, #drmaurino, #jÃ³ffily, #drjo...",have bbl authent expertnflag4 specialis vaserl...,quick recoveri result 10 day maurinojoffili dr...,Dr Maurino Joffily
1,30,2.760000e+18,http://www.drjoffily.co.uk/,23300.0,276.0,7003.0,maurino_joffily,https://instagram.fwzy1-2.fna.fbcdn.net/v/t51....,388.0,1.640000e+12,https://instagram.fuvf1-1.fna.fbcdn.net/v/t51....,https://www.instagram.com/p/CZUrYFrOO5v/,24.0,"[{""src"":""https://instagram.fuvf1-1.fna.fbcdn.n...",maurino_joffily,"[#maurinojoffily, #drmaurino, #drjoffily, #drj...",have bbl authent expertnflag4 specialis vaserl...,thi patient happi chang bodi total new figur a...,Dr Maurino Joffily
2,31,2.760000e+18,http://www.drjoffily.co.uk/,23300.0,276.0,7003.0,maurino_joffily,https://instagram.fwzy1-2.fna.fbcdn.net/v/t51....,160.0,1.640000e+12,https://instagram.fuvf1-1.fna.fbcdn.net/v/t51....,https://www.instagram.com/p/CZAgVoyPi0i/,8.0,"[{""src"":""https://instagram.fuvf1-1.fna.fbcdn.n...",maurino_joffily,"[#maurino, #joffily, #joffilyvaser, #drmaurino...",have bbl authent expertnflag4 specialis vaserl...,360 vaser lipo fat transfer correct hip dip ma...,Dr Maurino Joffily
3,32,2.760000e+18,http://www.drjoffily.co.uk/,23300.0,276.0,7003.0,maurino_joffily,https://instagram.fwzy1-2.fna.fbcdn.net/v/t51....,211.0,1.640000e+12,https://instagram.fuvf1-1.fna.fbcdn.net/v/t51....,https://www.instagram.com/p/CY9pMGVP_BY/,24.0,"[{""src"":""https://instagram.fuvf1-1.fna.fbcdn.n...",maurino_joffily,"[#drjoffily, #maurino, #maurinojoffily, #vaser...",have bbl authent expertnflag4 specialis vaserl...,breast uplift without implant good combin 360 ...,Dr Maurino Joffily
4,33,2.750000e+18,http://www.drjoffily.co.uk/,23300.0,276.0,7003.0,maurino_joffily,https://instagram.fwzy1-2.fna.fbcdn.net/v/t51....,210.0,1.640000e+12,https://instagram.fuvf1-1.fna.fbcdn.net/v/t51....,https://www.instagram.com/p/CYrbwXQrY3v/,15.0,"[{""src"":""https://instagram.fuvf1-1.fna.fbcdn.n...",maurino_joffily,"[#drmaurino, #maurino, #drjoffily, #vaserlipo,...",have bbl authent expertnflag4 specialis vaserl...,great transform combin procedur drmaurino maur...,Dr Maurino Joffily
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58924,62916,1.630000e+18,https://linktr.ee/drallenrezai,155118.0,776.0,376.0,drallenrezai,https://scontent-iev1-1.cdninstagram.com/v/t51...,102.0,1.510000e+12,https://instagram.ftas2-1.fna.fbcdn.net/v/t51....,https://www.instagram.com/p/BawCgfjDtqX/,1.0,"[{""src"":""https://instagram.ftas2-1.fna.fbcdn.n...",drallenrezai,[],consult reconstruct firm believ dpointright ðÿ...,see good peopl situat life alway beauti journey,Allen Rezai MD
58925,62917,1.630000e+18,https://linktr.ee/drallenrezai,155118.0,776.0,376.0,drallenrezai,https://scontent-iev1-1.cdninstagram.com/v/t51...,131.0,1.510000e+12,https://instagram.ftas1-2.fna.fbcdn.net/v/t51....,https://www.instagram.com/p/BaqhugBDQvU/,6.0,"[{""src"":""https://instagram.ftas1-2.fna.fbcdn.n...",drallenrezai,[],consult reconstruct firm believ dpointright ðÿ...,expatwoman nese opportun tri prp skin rejuven ...,Allen Rezai MD
58926,62918,1.630000e+18,https://linktr.ee/drallenrezai,155118.0,776.0,376.0,drallenrezai,https://scontent-iev1-1.cdninstagram.com/v/t51...,115.0,1.510000e+12,https://instagram.ftas1-2.fna.fbcdn.net/v/t51....,https://www.instagram.com/p/BalNRWQjn7g/,1.0,"[{""src"":""https://instagram.ftas1-2.fna.fbcdn.n...",drallenrezai,[],consult reconstruct firm believ dpointright ðÿ...,the decis breas

In [69]:
df_final

,Unnamed: 0,id,external_url,followers,posts_count,following,account,profile_image_link,likes,datetime,image_url,url,comments,thumbnails,input_account,hashtags,biography_final,caption_final,profile_final
0,29,2.770000e+18,http://www.drjoffily.co.uk/,23300.0,276.0,7003.0,maurino_joffily,https://instagram.fwzy1-2.fna.fbcdn.net/v/t51....,173.0,1.640000e+12,https://instagram.fuvf1-1.fna.fbcdn.net/v/t51....,https://www.instagram.com/p/CZki1wfvXYq/,17.0,"[{""src"":""https://instagram.fuvf1-1.fna.fbcdn.n...",maurino_joffily,"[#maurinojoffily, #drmaurino, #jÃ³ffily, #drjo...",have bbl authent expertnflag4 specialis vaserl...,quick recoveri result 10 day maurinojoffili dr...,Dr Maurino Joffily
1,30,2.760000e+18,http://www.drjoffily.co.uk/,23300.0,276.0,7003.0,maurino_joffily,https://instagram.fwzy1-2.fna.fbcdn.net/v/t51....,388.0,1.640000e+12,https://instagram.fuvf1-1.fna.fbcdn.net/v/t51....,https://www.instagram.com/p/CZUrYFrOO5v/,24.0,"[{""src"":""https://instagram.fuvf1-1.fna.fbcdn.n...",maurino_joffily,"[#maurinojoffily, #drmaurino, #drjoffily, #drj...",have bbl authent expertnflag4 specialis vaserl...,thi patient happi chang bodi total new figur a...,Dr Maurino Joffily
2,31,2.760000e+18,http://www.drjoffily.co.uk/,23300.0,276.0,7003.0,maurino_joffily,https://instagram.fwzy1-2.fna.fbcdn.net/v/t51....,160.0,1.640000e+12,https://instagram.fuvf1-1.fna.fbcdn.net/v/t51....,https://www.instagram.com/p/CZAgVoyPi0i/,8.0,"[{""src"":""https://instagram.fuvf1-1.fna.fbcdn.n...",maurino_joffily,"[#maurino, #joffily, #joffilyvaser, #drmaurino...",have bbl authent expertnflag4 specialis vaserl...,360 vaser lipo fat transfer correct hip dip ma...,Dr Maurino Joffily
3,32,2.760000e+18,http://www.drjoffily.co.uk/,23300.0,276.0,7003.0,maurino_joffily,https://instagram.fwzy1-2.fna.fbcdn.net/v/t51....,211.0,1.640000e+12,https://instagram.fuvf1-1.fna.fbcdn.net/v/t51....,https://www.instagram.com/p/CY9pMGVP_BY/,24.0,"[{""src"":""https://instagram.fuvf1-1.fna.fbcdn.n...",maurino_joffily,"[#drjoffily, #maurino, #maurinojoffily, #vaser...",have bbl authent expertnflag4 specialis vaserl...,breast uplift without implant good combin 360 ...,Dr Maurino Joffily
4,33,2.750000e+18,http://www.drjoffily.co.uk/,23300.0,276.0,7003.0,maurino_joffily,https://instagram.fwzy1-2.fna.fbcdn.net/v/t51....,210.0,1.640000e+12,https://instagram.fuvf1-1.fna.fbcdn.net/v/t51....,https://www.instagram.com/p/CYrbwXQrY3v/,15.0,"[{""src"":""https://instagram.fuvf1-1.fna.fbcdn.n...",maurino_joffily,"[#drmaurino, #maurino, #drjoffily, #vaserlipo,...",have bbl authent expertnflag4 specialis vaserl...,great transform combin procedur drmaurino maur...,Dr Maurino Joffily
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58924,62916,1.630000e+18,https://linktr.ee/drallenrezai,155118.0,776.0,376.0,drallenrezai,https://scontent-iev1-1.cdninstagram.com/v/t51...,102.0,1.510000e+12,https://instagram.ftas2-1.fna.fbcdn.net/v/t51....,https://www.instagram.com/p/BawCgfjDtqX/,1.0,"[{""src"":""https://instagram.ftas2-1.fna.fbcdn.n...",drallenrezai,[],consult reconstruct firm believ dpointright ðÿ...,see good peopl situat life alway beauti journey,Allen Rezai MD
58925,62917,1.630000e+18,https://linktr.ee/drallenrezai,155118.0,776.0,376.0,drallenrezai,https://scontent-iev1-1.cdninstagram.com/v/t51...,131.0,1.510000e+12,https://instagram.ftas1-2.fna.fbcdn.net/v/t51....,https://www.instagram.com/p/BaqhugBDQvU/,6.0,"[{""src"":""https://instagram.ftas1-2.fna.fbcdn.n...",drallenrezai,[],consult reconstruct firm believ dpointright ðÿ...,expatwoman nese opportun tri prp skin rejuven ...,Allen Rezai MD
58926,62918,1.630000e+18,https://linktr.ee/drallenrezai,155118.0,776.0,376.0,drallenrezai,https://scontent-iev1-1.cdninstagram.com/v/t51...,115.0,1.510000e+12,https://instagram.ftas1-2.fna.fbcdn.net/v/t51....,https://www.instagram.com/p/BalNRWQjn7g/,1.0,"[{""src"":""https://instagram.ftas1-2.fna.fbcdn.n...",drallenrezai,[],consult reconstruct firm believ dpointright ðÿ...,the decis breas

In [70]:
df_final.drop(['account'], axis=1, inplace=True)
df_final.drop(['Unnamed: 0'], axis=1, inplace=True)

In [71]:
# old_names = [ 'input_account', 'biography_final', 'caption_final', 'profile_final'] 
# new_names = {'account', 'biography', 'caption', 'profile'}
# df_final.columns=new_names


df = df_final.rename(columns={'input_account': 'account','biography_final':'biography','caption_final':'caption','profile_final':'profile'}) 

In [72]:
df

,id,external_url,followers,posts_count,following,profile_image_link,likes,datetime,image_url,url,comments,thumbnails,account,hashtags,biography,caption,profile
0,2.770000e+18,http://www.drjoffily.co.uk/,23300.0,276.0,7003.0,https://instagram.fwzy1-2.fna.fbcdn.net/v/t51....,173.0,1.640000e+12,https://instagram.fuvf1-1.fna.fbcdn.net/v/t51....,https://www.instagram.com/p/CZki1wfvXYq/,17.0,"[{""src"":""https://instagram.fuvf1-1.fna.fbcdn.n...",maurino_joffily,"[#maurinojoffily, #drmaurino, #jÃ³ffily, #drjo...",have bbl authent expertnflag4 specialis vaserl...,quick recoveri result 10 day maurinojoffili dr...,Dr Maurino Joffily
1,2.760000e+18,http://www.drjoffily.co.uk/,23300.0,276.0,7003.0,https://instagram.fwzy1-2.fna.fbcdn.net/v/t51....,388.0,1.640000e+12,https://instagram.fuvf1-1.fna.fbcdn.net/v/t51....,https://www.instagram.com/p/CZUrYFrOO5v/,24.0,"[{""src"":""https://instagram.fuvf1-1.fna.fbcdn.n...",maurino_joffily,"[#maurinojoffily, #drmaurino, #drjoffily, #drj...",have bbl authent expertnflag4 specialis vaserl...,thi patient happi chang bodi total new figur a...,Dr Maurino Joffily
2,2.760000e+18,http://www.drjoffily.co.uk/,23300.0,276.0,7003.0,https://instagram.fwzy1-2.fna.fbcdn.net/v/t51....,160.0,1.640000e+12,https://instagram.fuvf1-1.fna.fbcdn.net/v/t51....,https://www.instagram.com/p/CZAgVoyPi0i/,8.0,"[{""src"":""https://instagram.fuvf1-1.fna.fbcdn.n...",maurino_joffily,"[#maurino, #joffily, #joffilyvaser, #drmaurino...",have bbl authent expertnflag4 specialis vaserl...,360 vaser lipo fat transfer correct hip dip ma...,Dr Maurino Joffily
3,2.760000e+18,http://www.drjoffily.co.uk/,23300.0,276.0,7003.0,https://instagram.fwzy1-2.fna.fbcdn.net/v/t51....,211.0,1.640000e+12,https://instagram.fuvf1-1.fna.fbcdn.net/v/t51....,https://www.instagram.com/p/CY9pMGVP_BY/,24.0,"[{""src"":""https://instagram.fuvf1-1.fna.fbcdn.n...",maurino_joffily,"[#drjoffily, #maurino, #maurinojoffily, #vaser...",have bbl authent expertnflag4 specialis vaserl...,breast uplift without implant good combin 360 ...,Dr Maurino Joffily
4,2.750000e+18,http://www.drjoffily.co.uk/,23300.0,276.0,7003.0,https://instagram.fwzy1-2.fna.fbcdn.net/v/t51....,210.0,1.640000e+12,https://instagram.fuvf1-1.fna.fbcdn.net/v/t51....,https://www.instagram.com/p/CYrbwXQrY3v/,15.0,"[{""src"":""https://instagram.fuvf1-1.fna.fbcdn.n...",maurino_joffily,"[#drmaurino, #maurino, #drjoffily, #vaserlipo,...",have bbl authent expertnflag4 specialis vaserl...,great transform combin procedur drmaurino maur...,Dr Maurino Joffily
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58924,1.630000e+18,https://linktr.ee/drallenrezai,155118.0,776.0,376.0,https://scontent-iev1-1.cdninstagram.com/v/t51...,102.0,1.510000e+12,https://instagram.ftas2-1.fna.fbcdn.net/v/t51....,https://www.instagram.com/p/BawCgfjDtqX/,1.0,"[{""src"":""https://instagram.ftas2-1.fna.fbcdn.n...",drallenrezai,[],consult reconstruct firm believ dpointright ðÿ...,see good peopl situat life alway beauti journey,Allen Rezai MD
58925,1.630000e+18,https://linktr.ee/drallenrezai,155118.0,776.0,376.0,https://scontent-iev1-1.cdninstagram.com/v/t51...,131.0,1.510000e+12,https://instagram.ftas1-2.fna.fbcdn.net/v/t51....,https://www.instagram.com/p/BaqhugBDQvU/,6.0,"[{""src"":""https://instagram.ftas1-2.fna.fbcdn.n...",drallenrezai,[],consult reconstruct firm believ dpointright ðÿ...,expatwoman nese opportun tri prp skin rejuven ...,Allen Rezai MD
58926,1.630000e+18,https://linktr.ee/drallenrezai,155118.0,776.0,376.0,https://scontent-iev1-1.cdninstagram.com/v/t51...,115.0,1.510000e+12,https://instagram.ftas1-2.fna.fbcdn.net/v/t51....,https://www.instagram.com/p/BalNRWQjn7g/,1.0,"[{""src"":""https://instagram.ftas1-2.fna.fbcdn.n...",drallenrezai,[],consult reconstruct firm believ dpointright ðÿ...,the decis breast implant major step woman extr...,Allen Rezai MD
58927,1.630000e+18,https://linktr.ee/drallenrezai,155118.0,776.0,376.0,https://scontent-iev1-1.cdninstagram.com/v/t51...,133.0,1.510000e+12,https://insta

In [73]:
df.to_csv('Instragram_after_Preprocessing.csv', encoding='utf-8')